In [1]:
import pandas as pd
from pymongo import MongoClient

In [2]:

# -----------------------------
# CONFIG
# -----------------------------
CSV_PATH = "pharmacy_combined.csv"   # path to your CSV file
MONGO_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "sentiment_db"
COLLECTION_NAME = "total_Reviews"   # MUST match your DataIngestion constant

In [3]:



# -----------------------------
# LOAD CSV
# -----------------------------
df = pd.read_csv(CSV_PATH)

# Basic sanity check
required_columns = {"text", "intent", "topic", "sentiment"}
missing = required_columns - set(df.columns)

if missing:
    raise ValueError(f"CSV is missing columns: {missing}")

# Convert DataFrame to dict records
records = df.to_dict(orient="records")

# -----------------------------
# CONNECT TO MONGO
# -----------------------------
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]


In [4]:

# -----------------------------
# INSERT DATA
# -----------------------------
if records:
    result = collection.insert_many(records)
    print(f"✅ Inserted {len(result.inserted_ids)} records into MongoDB")
else:
    print("⚠️ No records found in CSV")

# -----------------------------
# VERIFY INSERTION
# -----------------------------
print("📊 Sample documents:")
for doc in collection.find().limit(3):
    doc.pop("_id", None)
    print(doc)

client.close()


✅ Inserted 2284 records into MongoDB
📊 Sample documents:
{'text': 'الصيدلية دي أحسن صيدلية شفتها! كل المنتجات موجودة وبسعر حلو. شكرًا لكم!', 'intent': 'Praise', 'topic': 'availability', 'sentiment': 'positive'}
{'text': 'أنا أحب الصيدلية دي كتير! كل مرة بجيلها لاقى كل اللي محتاجه. شكرًا لكم على الجهد.', 'intent': 'Praise', 'topic': 'availability', 'sentiment': 'positive'}
{'text': 'الصيدلية دي جوه عيني! دايماً متوفرة على كل اللي نحتاجه. شكرًا لكم على المجهود.', 'intent': 'Praise', 'topic': 'availability', 'sentiment': 'positive'}
